In [2]:
import os
import cv2
import time
import numpy as np
import pyrealsense2 as rs
import matplotlib.pyplot as plt

# import tensorflow as tf
# assert tf.__version__.startswith('2')

import mediapipe as mp

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles


2023-06-07 18:05:04.111433: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 18:05:04.216084: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guillaume/anaconda3/envs/semproj/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda-11.7/lib64:
2023-06-07 18:05:04.216100: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-07 18:05:04.810691: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not 

In [3]:
def get_hand_coordinates(pose_landmarks):
    left_hand_coords = {"max_x": -1000, "max_y": -1000, "min_x": 1000, "min_y": 1000}
    right_hand_coords = {"max_x": -1000, "max_y": -1000, "min_x": 1000, "min_y": 1000}

    left_hand_index = [17, 19, 21] # pinky index thumb
    right_hand_index = [18, 20, 22]

    # Extract the coordinates of the left and right hands
    for idx, landmark in enumerate(pose_landmarks.landmark):
        if idx in left_hand_index:
              # Left hand
            if landmark.x > left_hand_coords["max_x"]:
                left_hand_coords["max_x"] = landmark.x
            if landmark.y > left_hand_coords["max_y"]:
                left_hand_coords["max_y"] = landmark.y
            if landmark.x < left_hand_coords["min_x"]:
                left_hand_coords["min_x"] = landmark.x
            if landmark.y < left_hand_coords["min_y"]:
                left_hand_coords["min_y"] = landmark.y

        elif idx in right_hand_index:  # Right hand

            if landmark.x > right_hand_coords["max_x"]:
                right_hand_coords["max_x"] = landmark.x
            if landmark.y > right_hand_coords["max_y"]:
                right_hand_coords["max_y"] = landmark.y
            if landmark.x < right_hand_coords["min_x"]:
                right_hand_coords["min_x"] = landmark.x
            if landmark.y < right_hand_coords["min_y"]:
                right_hand_coords["min_y"] = landmark.y


    if any(item < 0 or item > 1 for item in left_hand_coords.values()): # coordinates are out of the frame
        left_hand_coords = None
    if any(item <0 or item > 1 for item in right_hand_coords.values()):
        right_hand_coords = None

    return right_hand_coords, left_hand_coords # we have to switch them here due to the cv2.flip

def crop_hand(frame, hand_coords):
    # image_height, image_width = frame.height, frame.width
    
    image_height, image_width, _ = frame.shape

    margin = 80

    x_min = max(0, int(hand_coords["min_x"] * image_width) - margin)
    y_min = max(0, int(hand_coords["min_y"] * image_height) - margin)
    x_max = min(image_width - 1, int(hand_coords["max_x"] * image_width) + margin)
    y_max = min(image_height - 1, int(hand_coords["max_y"] * image_height) + margin)

    # Crop the image around the hand region
    hand_image = frame[y_min:y_max, x_min:x_max]

    return hand_image

In [4]:
def classify_hand(mp_image):

        # mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data = hand_image)
        # mp_image = hand_image
        result = recognizer.recognize(mp_image) 
    
        try:
            prediction = result.gestures[0][0].category_name
            confidence = result.gestures[0][0].score          
        except:
            prediction = "no hands hee"
            confidence = ""

        return prediction, confidence

## Realsense

In [5]:

pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)

device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))
found_rgb = False

for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

# we enable rgb stream 
if device_product_line == 'L500':
    config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, 30)
else:
    config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)

# we enable depth stream
config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)

# create colorizer for the depth image
colorizer = rs.colorizer()

In [163]:
def get_wrist_depth(aligned_depth_image, pose_results):

    debug = False
    
    height = aligned_depth_image.shape[0]
    width = aligned_depth_image.shape[1]
    right_wrist = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]
    right_index = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX]
    right_pinky = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_PINKY]
    right_thumb = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_THUMB]
    
    keypoints = [right_wrist, right_index, right_pinky, right_thumb]
    xs, ys = [], []
    for keypoint in keypoints:
        xs.append(keypoint.x)
        ys.append(keypoint.y)

    x = np.mean(xs)
    y = np.mean(ys) 
    
    x,y = int(x*width), int(y*height)

    square_size = 5
    x_min = max(0, x-square_size)
    y_min = max(0, y-square_size)
    x_max = min(height-1, x+square_size)
    y_max = min(width-1, y+square_size)

    x = int(right_wrist.x*width)
    y = int(right_wrist.y*height)

    if debug:
        print(f"x {x}, y {y}, width {width}, height {height}")
        print(f"wrist depth {wrist_depth}, x {right_wrist.x}, y {right_wrist.y}")

    wrist_depth = aligned_depth_image[int(right_wrist.y*height), int(right_wrist.x*width)]
    

    return wrist_depth

In [164]:
def get_body_depth(aligned_depth_frame, pose_results):

    height = aligned_depth_frame.shape[0]
    width = aligned_depth_frame.shape[1]
    right_shoulder = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
    left_shoulder = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
    right_hip = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]
    left_hip = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]

    keypoints = [right_shoulder, left_shoulder, right_hip, left_hip]
    xs, ys = [], []
    for keypoint in keypoints:
        xs.append(keypoint.x)
        ys.append(keypoint.y)

    x,y = np.mean(xs), np.mean(ys)
    x,y = int(x*width), int(y*height)

    square_size = 5
    x_min = max(0, x-square_size)
    y_min = max(0, y-square_size)
    x_max = min(height-1, x+square_size)
    y_max = min(width-1, y+square_size)

    body_depth = np.mean(aligned_depth_frame[y_min:y_max, x_min:x_max].astype(float))
    print(f"body depth {body_depth}")

    return body_depth


In [177]:
pipeline.stop()
cv2.destroyAllWindows()

In [179]:
pipeline.start(config)

new_frame_time = 0
prev_frame_time = 0

cv2.namedWindow('Webcam Feed', cv2.WINDOW_NORMAL)

BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a gesture recognizer instance with the image mode:
options = GestureRecognizerOptions(
    base_options=BaseOptions(model_asset_path='exported_model_4/gesture_recognizer.task'),
    running_mode=VisionRunningMode.IMAGE,
    num_hands = 1,
    min_hand_detection_confidence=0.2,
    min_hand_presence_confidence=0.2)
with GestureRecognizer.create_from_options(options) as recognizer:
    with mp_pose.Pose(
        model_complexity=1,
        min_detection_confidence=0.6,
        min_tracking_confidence=0.6) as pose:    
        while True:

            frameset = pipeline.wait_for_frames()
            color_frame = frameset.get_color_frame()
            depth_frame = frameset.get_depth_frame()

            # Create alignment primitive with color as its target stream:
            align = rs.align(rs.stream.color)
            frameset = align.process(frameset)

            # Update color and depth frames:
            aligned_depth_frame = frameset.get_depth_frame()
            colorized_depth = np.asanyarray(colorizer.colorize(aligned_depth_frame).get_data())

            if not color_frame:
                continue

            color_image = np.asarray(color_frame.get_data())
            frame = color_image
            # frame = cv2.flip(color_image,1)

            #hands detection and classification
            pose_results = pose.process(frame) # find body keypoints

            right_pred = "no hands"
            right_conf = ""

            hand_in_front = False

            if pose_results.pose_landmarks is not None:
                # Get the coordinates of the left and right hands
                right_hand_coords, _  = get_hand_coordinates(pose_results.pose_landmarks)
                
                if right_hand_coords is not None:
                    right_hand_image = (crop_hand(frame, right_hand_coords))
                    # right_hand_image = cv2.flip(right_hand_image,1)
                    # right_hand_image = cv2.flip(right_hand_image,1)
                    right_hand_image = cv2.cvtColor(right_hand_image, cv2.COLOR_BGR2RGB)
                    right_hand_image = mp.Image(image_format=mp.ImageFormat.SRGB, data = right_hand_image)
                    right_pred, right_conf = classify_hand(right_hand_image)
                    

                    # Draw the bounding box and prediction on the frame
                    margin_x = 80
                    margin_y = int(margin_x * 1.25)
                    image_height, image_width, _ = frame.shape
                    x_min = max(0, int(right_hand_coords["min_x"] * image_width) - margin_x)
                    y_min = max(0, int(right_hand_coords["min_y"] * image_height) - margin_y)
                    x_max = min(image_width - 1, int(right_hand_coords["max_x"] * image_width) + margin_x)
                    y_max = min(image_height - 1, int(right_hand_coords["max_y"] * image_height) + margin_y)
                    start_box = (x_min, y_min)
                    end_box = (x_max, y_max)
                    frame = cv2.rectangle(frame, start_box, end_box, (255,0,0), 2)
                    cv2.putText(frame, right_pred, start_box, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

                    # check if the hand is in front of the body
                    wrist_depth = get_wrist_depth(np.asanyarray(aligned_depth_frame.get_data()), pose_results)
                    body_depth = get_body_depth(np.asanyarray(aligned_depth_frame.get_data()), pose_results)

                    if body_depth - wrist_depth > 250:
                        hand_in_front = True

                    # annotated_image = draw_landmarks_on_image(frame, result)

                    mp_drawing.draw_landmarks(
                    frame,
                    pose_results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

            new_frame_time = time.time()
            fps = 1/(new_frame_time-prev_frame_time)
            prev_frame_time = new_frame_time

            annotated_image = frame
            # cv2.putText(annotated_image, f"{wrist_depth}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            # Write fps on the frame
            cv2.putText(annotated_image, f"Hand in front: {hand_in_front}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(annotated_image, f"FPS: {fps}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            
            # images = np.hstack((annotated_image, colorized_depth))
            # images = np.hstack((annotated_image, new_img))
            
            # Display the frame in a window named 'Webcam Feed'
            cv2.imshow('Webcam Feed', annotated_image)
            
            # Exit the loop if the 'q' key is pressed
            if cv2.waitKey(1) == 27:  # 27 is the ASCII code for the Escape key
                break

# Release the webcam and close the window
pipeline.stop()
cv2.destroyAllWindows()

W20230607 19:40:12.536485 43431 gesture_recognizer_graph.cc:128] Hand Gesture Recognizer contains CPU only ops. Sets HandGestureRecognizerGraph acceleration to Xnnpack.


body depth 1215.2
body depth 1205.1
body depth 1199.97
body depth 1193.99
body depth 1193.71
body depth 1192.65
body depth 1192.82
body depth 1190.2
body depth 1188.52
body depth 1188.23
body depth 1181.11
body depth 1178.39
body depth 1178.12
body depth 1164.27
body depth 1165.31
body depth 1177.25
body depth 1178.48
body depth 1182.35
body depth 1183.67
body depth 1179.39
body depth 1177.7
body depth 1178.02
body depth 1176.53
body depth 1177.34
body depth 1178.57
body depth 1166.95
body depth 1182.1
body depth 1179.75
body depth 1182.92
body depth 1185.12
body depth 1185.91
body depth 1184.56
body depth 1181.38
body depth 1181.0
body depth 1179.28
body depth 1178.59
body depth 1175.65
body depth 1175.61
body depth 1174.2
body depth 1173.37
body depth 1161.6
body depth 1173.5
body depth 1128.65
body depth 1147.5
body depth 1147.65
body depth 1161.78
body depth 1189.15
body depth 1192.76
body depth 1187.51
body depth 1172.2
body depth 1161.47
body depth 1163.49
body depth 1162.0
body 

/home/guillaume/anaconda3/envs/semproj/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/guillaume/anaconda3/envs/semproj/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth nan
body depth 1515.67
body depth 1442.91
body depth 1369.38
body depth 1309.2
body depth 1075.99
body depth 980.97
body depth 917.59
body depth 836.5
body depth 718.28
body depth 340.06
body depth 240.97
body depth 725.61
body depth 832.04
body depth 925.32
body depth 972.16
body depth 1020.65
body depth 1068.11
body depth 1104.99
body depth 1093.33
body depth 1164.12
body depth 1226.2
body depth 107

In [173]:
print(pose_results.pose_landmarks)

None


## Webcam

In [16]:
cap = cv2.VideoCapture(0)

new_frame_time = 0
prev_frame_time = 0

cv2.namedWindow('Webcam Feed', cv2.WINDOW_NORMAL)

BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a gesture recognizer instance with the image mode:
options = GestureRecognizerOptions(
    base_options=BaseOptions(model_asset_path='exported_model_3/gesture_recognizer.task'),
    running_mode=VisionRunningMode.IMAGE,
    num_hands = 1,
    min_hand_detection_confidence=0.2,
    min_hand_presence_confidence=0.2)
with GestureRecognizer.create_from_options(options) as recognizer:
    with mp_pose.Pose(
        model_complexity=1,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:    
        while True:

            ret, color_frame = cap.read()
            
            if not ret:
                print("Failed to read the frame.")
                break
            
            frame = cv2.flip(color_frame,1)

            #hands detection and classification
            pose_results = pose.process(frame) # find body keypoints

            right_pred = "no hands"
            right_conf = ""

            if pose_results.pose_landmarks:
                # Get the coordinates of the left and right hands
                _, right_hand_coords = get_hand_coordinates(pose_results.pose_landmarks)
                
                if right_hand_coords is not None:
                    right_hand_image = (crop_hand(frame, right_hand_coords))
                    # right_hand_image = cv2.flip(right_hand_image,1)
                    # right_hand_image = cv2.flip(right_hand_image,1)
                    right_hand_image = cv2.cvtColor(right_hand_image, cv2.COLOR_BGR2RGB)
                    right_hand_image = mp.Image(image_format=mp.ImageFormat.SRGB, data = right_hand_image)
                    right_pred, right_conf = classify_hand(right_hand_image)
                    

                    # Draw the bounding box and prediction on the frame
                    margin_x = 80
                    margin_y = int(margin_x * 1.25)
                    image_height, image_width, _ = frame.shape
                    x_min = max(0, int(right_hand_coords["min_x"] * image_width) - margin_x)
                    y_min = max(0, int(right_hand_coords["min_y"] * image_height) - margin_y)
                    x_max = min(image_width - 1, int(right_hand_coords["max_x"] * image_width) + margin_x)
                    y_max = min(image_height - 1, int(right_hand_coords["max_y"] * image_height) + margin_y)
                    start_box = (x_min, y_min)
                    end_box = (x_max, y_max)
                    frame = cv2.rectangle(frame, start_box, end_box, (255,0,0), 2)
                    cv2.putText(frame, right_pred, start_box, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            # annotated_image = draw_landmarks_on_image(frame, result)
            annotated_image = frame

            mp_drawing.draw_landmarks(
            annotated_image,
            pose_results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

            new_frame_time = time.time()
            fps = 1/(new_frame_time-prev_frame_time)
            prev_frame_time = new_frame_time

            cv2.putText(annotated_image, f"{pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].z}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Write fps on the frame
            cv2.putText(annotated_image, f"FPS: {fps}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # Display the frame in a window named 'Webcam Feed'
            cv2.imshow('Webcam Feed', annotated_image)
            
            # Exit the loop if the 'q' key is pressed
            if cv2.waitKey(1) == 27:  # 27 is the ASCII code for the Escape key
                break

# Release the webcam and close the window
pipeline.stop()
cv2.destroyAllWindows()

W20230607 16:11:43.663633 23045 gesture_recognizer_graph.cc:128] Hand Gesture Recognizer contains CPU only ops. Sets HandGestureRecognizerGraph acceleration to Xnnpack.


AttributeError: 'NoneType' object has no attribute 'landmark'

In [22]:
cv2.destroyAllWindows()
cap.release()

x: 0.09573205560445786
y: 1.6362898349761963
z: -0.9263961911201477
visibility: 0.09073561429977417